In [16]:
from utils import TestResults,TrainSplit
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, log_loss

import numpy as np
import optuna

In [17]:
X_train, X_test, y_train, y_test = TrainSplit().get_split()

Loading the data
Categorial Feature Imputed


NUM IMPUTE: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 47.93it/s]


Spliting the Data based on time
X_train:(3387880, 80), X_test:(97972, 80) , y_train:(3387880, 2) , y_test:(97972, 2)


In [18]:
cat_feat = TrainSplit.CATEGORIES + TrainSplit.BINARY
num_feat = TrainSplit.NUMERICAL
all_feat = cat_feat + num_feat

In [4]:
import pandas as pd
X = pd.concat([X_train,X_test],axis=0)

In [6]:
X[all_feat]

,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,...,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79
0,30131,7152,16170,25604,25613,27941,21218,869,22970,11810,...,0.672952,0.000000,3.426729,0.571121,0.115692,1.156922,0.269948,0.0,0.000000,0.0
1,20095,563,22861,25604,22651,27941,19203,869,19343,28763,...,0.000000,0.000000,0.571121,0.571121,0.115692,1.156922,0.269948,0.0,0.000000,0.0
2,20095,563,22861,25604,21280,27941,21621,23218,22970,12213,...,0.000305,0.000000,0.000000,0.000000,0.000000,1.156922,0.038564,0.0,0.000000,0.0
3,17758,22294,29040,25604,15836,27941,21218,21533,22970,28360,...,1.046330,0.571121,5.711215,2.284486,0.115692,1.156922,0.269948,0.0,0.000000,0.0
4,11077,7152,18575,15908,16861,27941,21218,9638,19343,11407,...,0.658138,0.000000,1.142243,0.000000,0.038564,1.156922,0.269948,0.0,37.384575,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485578,20095,563,31686,21545,590,27941,19203,6675,22970,30251,...,0.000204,0.000000,0.571121,0.571121,0.038564,1.156922,0.077128,0.0,0.000000,0.0
3485664,11077,7152,18575,21545,32377,27941,21218,6675,19343,11810,...,0.750122,0.000000,0.571121,0.571121,0.115692,1.156922,0.269948,0.0,0.000000,0.0
3485724,20095,563,31686,21545,590,27941,18800,6675,22970,29166,...,0.000204,0.000000,1.142243,0.000000,0.115692,1.156922,0.269948,0.0,0.000000,0.0
3485743,26325,7779,21563,25604,29287,27941,21218,14659,19343,31181,...,2.177379,0.000000,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.000000,0.0


In [9]:
y = pd.concat([y_train,y_test],axis=0)

In [15]:
def objective(trail):
    params = {
        'max_depth':trail.suggest_int('max_depth',3,8),
        'learning_rate':trail.suggest_float('learning_rate',0.05,0.3),
        'n_estimators':trail.suggest_int('n_estimators',100,650),
        'tree_method':'gpu_hist',
        'objective':'binary:logistic'
    }
    target = TrainSplit.IS_INSTALLED[0]
    model = XGBClassifier(**params)
    model.fit(X_train[use_features],y_train[target])
    y_pred = model.predict(X_test[use_features])
    score = log_loss(y_test[target],y_pred)
    return score

In [14]:
params_click = {'max_depth': 4, 'learning_rate': 0.08751617649545007, 'n_estimators': 549}

In [6]:
use_features = cat_feat
study_install = optuna.create_study(direction='minimize',study_name='install_log_loss')
study_install.optimize(objective, n_trials=30)

[I 2023-05-24 03:32:45,567] A new study created in memory with name: install_log_loss
[I 2023-05-24 03:32:53,883] Trial 0 finished with value: 6.3915016938015645 and parameters: {'max_depth': 3, 'learning_rate': 0.08271949323619097, 'n_estimators': 304}. Best is trial 0 with value: 6.3915016938015645.
[I 2023-05-24 03:33:02,464] Trial 1 finished with value: 6.3188824055966135 and parameters: {'max_depth': 3, 'learning_rate': 0.1624819641702333, 'n_estimators': 407}. Best is trial 1 with value: 6.3188824055966135.
[I 2023-05-24 03:33:11,412] Trial 2 finished with value: 6.305488341793212 and parameters: {'max_depth': 5, 'learning_rate': 0.11920574126675722, 'n_estimators': 309}. Best is trial 2 with value: 6.305488341793212.
[I 2023-05-24 03:33:20,143] Trial 3 finished with value: 6.271646099116423 and parameters: {'max_depth': 5, 'learning_rate': 0.15740168923044817, 'n_estimators': 301}. Best is trial 3 with value: 6.271646099116423.
[I 2023-05-24 03:33:29,371] Trial 4 finished with v

In [15]:
use_features = num_feat
study_install = optuna.create_study(direction='minimize',study_name='install_log_loss')
study_install.optimize(objective, n_trials=30)

[I 2023-05-24 03:40:53,185] A new study created in memory with name: install_log_loss
[I 2023-05-24 03:41:03,844] Trial 0 finished with value: 6.315358649384043 and parameters: {'max_depth': 8, 'learning_rate': 0.28174304603908057, 'n_estimators': 340}. Best is trial 0 with value: 6.315358649384043.
[I 2023-05-24 03:41:10,920] Trial 1 finished with value: 6.270233575248056 and parameters: {'max_depth': 6, 'learning_rate': 0.17188265984298395, 'n_estimators': 237}. Best is trial 1 with value: 6.270233575248056.
[I 2023-05-24 03:41:15,117] Trial 2 finished with value: 6.287506152407862 and parameters: {'max_depth': 6, 'learning_rate': 0.25719682871806077, 'n_estimators': 101}. Best is trial 1 with value: 6.270233575248056.
[I 2023-05-24 03:41:27,760] Trial 3 finished with value: 6.26882451185115 and parameters: {'max_depth': 8, 'learning_rate': 0.20066101752937415, 'n_estimators': 407}. Best is trial 3 with value: 6.26882451185115.
[I 2023-05-24 03:41:39,561] Trial 4 finished with value:

In [ ]:
use_features = all_feat + TrainSplit.DATE
study_install = optuna.create_study(direction='minimize',study_name='install_log_loss')
study_install.optimize(objective, n_trials=30)

[I 2023-05-27 22:40:38,784] A new study created in memory with name: install_log_loss
[I 2023-05-27 22:40:49,453] Trial 0 finished with value: 6.391161668145922 and parameters: {'max_depth': 3, 'learning_rate': 0.09527659985945318, 'n_estimators': 297}. Best is trial 0 with value: 6.391161668145922.
[I 2023-05-27 22:41:00,586] Trial 1 finished with value: 6.522664016786587 and parameters: {'max_depth': 3, 'learning_rate': 0.08977111741299722, 'n_estimators': 349}. Best is trial 0 with value: 6.391161668145922.
[I 2023-05-27 22:41:09,480] Trial 2 finished with value: 6.573785178100373 and parameters: {'max_depth': 4, 'learning_rate': 0.15091973298191896, 'n_estimators': 120}. Best is trial 0 with value: 6.391161668145922.
[I 2023-05-27 22:41:25,317] Trial 3 finished with value: 6.949623560233651 and parameters: {'max_depth': 8, 'learning_rate': 0.1428115484562847, 'n_estimators': 305}. Best is trial 0 with value: 6.391161668145922.
[I 2023-05-27 22:41:36,367] Trial 4 finished with value

In [7]:
import pandas as pd
import numpy as np

test = pd.read_csv('../Data/test/000000000000.csv',sep='\t')
test['f_30'].fillna(X['f_30'].mode()[0],inplace=True)
test['f_31'].fillna(X['f_31'].mode()[0],inplace=True)
fmiss = "f_43,f_51,f_58,f_59,f_64,f_65,f_66,f_67,f_68,f_69,f_70".split(',')
for f in fmiss:
    test[f].fillna(X[f].mean(),inplace=True)

In [8]:
params_all_feat = {'max_depth': 7, 'learning_rate': 0.05198517520669243, 'n_estimators': 606,
                   'tree_method':'gpu_hist','objective':'binary:logistic'}
params_cat_feat = {'max_depth': 4, 'learning_rate': 0.2972844912008205, 'n_estimators': 257,
                   'tree_method':'gpu_hist','objective':'binary:logistic'}
params_num_feat = {'max_depth': 5, 'learning_rate': 0.18539658586808733, 'n_estimators': 603,
                   'tree_method':'gpu_hist','objective':'binary:logistic'}

In [10]:
model_all_feat = XGBClassifier(**params_all_feat)
model_all_feat.fit(X[all_feat],y[TrainSplit.IS_INSTALLED[0]])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05198517520669243,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=606, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [7]:
train_install_pred = model_all_feat.predict_proba(X_train[all_feat])[:,1]

In [8]:
X_train['xgb_pred'] = train_install_pred

In [11]:
X_test['xgb_pred'] = model_all_feat.predict_proba(X_test[all_feat])[:,1]

In [12]:
X_train['xgb_pred'],X_test['xgb_pred']

(0          0.068901
 1          0.242679
 2          0.279158
 3          0.128261
 4          0.028577
              ...   
 3485847    0.066495
 3485848    0.027818
 3485849    0.094219
 3485850    0.363060
 3485851    0.265923
 Name: xgb_pred, Length: 3387880, dtype: float32,
 12         0.161562
 20         0.302649
 26         0.025512
 83         0.074805
 97         0.029013
              ...   
 3485578    0.068327
 3485664    0.012814
 3485724    0.068605
 3485743    0.223019
 3485807    0.144830
 Name: xgb_pred, Length: 97972, dtype: float32)

In [22]:
log_loss(y_test[TrainSplit.IS_INSTALLED[0]],X_test['xgb_pred'])

0.36431280693160106

In [32]:
from sklearn.linear_model import LogisticRegression

def objective(trail):
    params = {  'C': trail.suggest_float('C',0,1),
                # 'class_weight':'balanced',# Try None as well
                'n_jobs': -1,
                'max_iter':2000
                # 'penalty': trail.suggest_categorical('penalty', ["l1", "l2", 'elasticnet']),
             }
    
    target = TrainSplit.IS_INSTALLED[0]
    model = LogisticRegression(**params)
    model.fit(X_train['xgb_pred'].values.reshape(-1,1),y_train[TrainSplit.IS_INSTALLED[0]])
    y_pred = model.predict_proba(X_test['xgb_pred'].values.reshape(-1,1))[:,1]
    score = log_loss(y_test[target],y_pred)
    return score

In [33]:
study_install = optuna.create_study(direction='minimize',study_name='logistic_loss')
study_install.optimize(objective, n_trials=30)

[I 2023-05-26 13:33:31,218] A new study created in memory with name: logistic_loss
[I 2023-05-26 13:33:38,473] Trial 0 finished with value: 0.377999565475483 and parameters: {'C': 0.16709935427365574}. Best is trial 0 with value: 0.377999565475483.
[I 2023-05-26 13:33:44,049] Trial 1 finished with value: 0.3779124085244122 and parameters: {'C': 0.033590909866925145}. Best is trial 1 with value: 0.3779124085244122.
[I 2023-05-26 13:33:49,691] Trial 2 finished with value: 0.3780174714463015 and parameters: {'C': 0.8530093830857242}. Best is trial 1 with value: 0.3779124085244122.
[I 2023-05-26 13:33:55,224] Trial 3 finished with value: 0.3780170242136945 and parameters: {'C': 0.7738685382490619}. Best is trial 1 with value: 0.3779124085244122.
[I 2023-05-26 13:34:00,823] Trial 4 finished with value: 0.3777659152655981 and parameters: {'C': 0.014049413490660911}. Best is trial 4 with value: 0.3777659152655981.
[I 2023-05-26 13:34:06,367] Trial 5 finished with value: 0.37801383674024214 an

KeyboardInterrupt: 

In [24]:
params = {  'C': 0.4436042554891082,
            'n_jobs': -1,
            'max_iter':2000}

model = LogisticRegression(**params)
model.fit(X_test['xgb_pred'].values.reshape(-1,1),y_test[TrainSplit.IS_INSTALLED[0]])    

LogisticRegression(C=0.4436042554891082, max_iter=2000, n_jobs=-1)

In [11]:
xgb_proba = model_all_feat.predict_proba(test[all_feat])

In [28]:
calib_proba = model.predict_proba(xgb_proba[:,1].reshape(-1,1))

In [14]:
TestResults(row_id=test['f_0'],is_click=np.random.random(xgb_proba[:,1].shape[0]),
            is_install=xgb_proba[:,1],model_name="xgb_full_data",config=params_all_feat)

Saved the test result to csv file as xgb_full_data_2023-05-27 22:29:12.147504.csv
Saved the model config to json file as xgb_full_data_2023-05-27 22:29:12.147504.json


In [38]:
np.abs(calib_proba[:,1]-xgb_proba[:,1]).mean()

0.03103035533591397

In [13]:
def objective(trail):
    params = {
        'max_depth':trail.suggest_int('max_depth',3,5),
        'learning_rate':trail.suggest_float('learning_rate',0.05,0.5),
        'n_estimators':trail.suggest_int('n_estimators',4,50),
        'tree_method':'gpu_hist',
        'objective':'binary:logistic'
    }
    # print(use_features)
    target = TrainSplit.IS_INSTALLED[0]
    model = XGBClassifier(**params)
    model.fit(X_train[use_features],y_train[target])
    y_pred = model.predict(X_test[use_features])
    score = log_loss(y_test[target],y_pred)
    # print(model.feature_names_in_)
    return score

In [14]:
use_features = all_feat+['xgb_pred']
study_install = optuna.create_study(direction='minimize',study_name='install_log_loss')
study_install.optimize(objective, n_trials=30)

[I 2023-05-25 14:41:13,397] A new study created in memory with name: install_log_loss
[I 2023-05-25 14:41:21,042] Trial 0 finished with value: 6.127142998691673 and parameters: {'max_depth': 5, 'learning_rate': 0.30359859436425907, 'n_estimators': 43}. Best is trial 0 with value: 6.127142998691673.
[I 2023-05-25 14:41:28,169] Trial 1 finished with value: 6.026658302034461 and parameters: {'max_depth': 4, 'learning_rate': 0.12269114208826071, 'n_estimators': 26}. Best is trial 1 with value: 6.026658302034461.
[I 2023-05-25 14:41:35,606] Trial 2 finished with value: 6.031946058340535 and parameters: {'max_depth': 4, 'learning_rate': 0.12097888238229475, 'n_estimators': 32}. Best is trial 1 with value: 6.026658302034461.
[I 2023-05-25 14:41:42,397] Trial 3 finished with value: 6.085183128531894 and parameters: {'max_depth': 3, 'learning_rate': 0.368858419347425, 'n_estimators': 28}. Best is trial 1 with value: 6.026658302034461.
[I 2023-05-25 14:41:48,707] Trial 4 finished with value: 5.9

In [16]:
level2xgb_params = {'max_depth': 3, 'learning_rate': 0.4900509143647155, 'n_estimators': 8,'tree_method':'gpu_hist',
                    'objective':'binary:logistic'}
target = TrainSplit.IS_INSTALLED[0]
model = XGBClassifier(**level2xgb_params)
model.fit(X_train[use_features],y_train[target])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.4900509143647155,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=8, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [19]:
import pandas as pd
test = pd.read_csv('../Data/test/000000000000.csv',sep='\t')
test['f_30'].fillna(X['f_30'].mode()[0],inplace=True)
test['f_31'].fillna(X['f_31'].mode()[0],inplace=True)
fmiss = "f_43,f_51,f_58,f_59,f_64,f_65,f_66,f_67,f_68,f_69,f_70".split(',')
for f in fmiss:
    test[f].fillna(X[f].mean(),inplace=True)

In [20]:
test['xgb_pred'] = model_all_feat.predict_proba(test[all_feat])[:,1]

In [21]:
install_pred_all = model.predict_proba(test[use_features])[:,1]

In [22]:
install_pred_all

array([0.20049362, 0.3117722 , 0.07335331, ..., 0.01842373, 0.11533546,
       0.05473805], dtype=float32)

In [28]:
np.mean(np.abs(install_pred_all - test['xgb_pred']))

0.01064414

In [8]:
# model_all_feat.classes_

In [24]:
install_pred_all.shape,test.shape

((160973,), (160973, 81))

In [33]:
# TestResults(row_id=test['f_0'],is_click=np.random.random(install_pred_all.shape[0]),
#             is_install=install_pred_all,model_name="xgb_all_feat_xgb_chain",config=params_all_feat)

In [29]:
model_cat_feat = XGBClassifier(**params_cat_feat)
model_cat_feat.fit(X_train[cat_feat],y_train[TrainSplit.IS_INSTALLED[0]])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2972844912008205,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=257, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [30]:
install_pred_cat = model_cat_feat.predict_proba(test[cat_feat])[:,1]

In [34]:
# TestResults(row_id=test['f_0'],is_click=np.random.random(install_pred_cat.shape[0]),
#             is_install=install_pred_cat,model_name="xgb_cat_feat",config=params_all_feat)

In [31]:
model_num_feat = XGBClassifier(**params_num_feat)
model_num_feat.fit(X_train[num_feat],y_train[TrainSplit.IS_INSTALLED[0]])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.18539658586808733,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=603, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [32]:
install_pred_num = model_num_feat.predict_proba(test[num_feat])[:,1]

In [35]:
# TestResults(row_id=test['f_0'],is_click=np.random.random(install_pred_num.shape[0]),
#             is_install=install_pred_num,model_name="xgb_num_feat",config=params_all_feat)

In [39]:
install_avg = (install_pred_cat + install_pred_num + test['xgb_pred'])/3

In [40]:
TestResults(row_id=test['f_0'],is_click=np.random.random(install_avg.shape[0]),
            is_install=install_avg,model_name="xgb_num_cat_all_avg",config=params_all_feat)

Saved the test result to csv file as xgb_num_cat_all_avg_2023-05-25 16:21:55.184468.csv
Saved the model config to json file as xgb_num_cat_all_avg_2023-05-25 16:21:55.184468.json


In [41]:
TrainSplit.get_config_list()

['catboost_all_feat_2023-05-25 11:41:56.839795.json',
 'xgb_stack_all_cat_num_2023-05-25 04:08:18.733781.json',
 'xgb_cat_feat_2023-05-24 05:09:00.493398.json',
 'xgb_num_feat_2023-05-24 05:09:31.486438.json',
 'xgb_stack_all_cat_num_xgb_2023-05-25 06:19:35.109117.json',
 'xgb_num_cat_all_avg_2023-05-25 16:21:55.184468.json',
 'xgb_all_feat_xgb_chain_2023-05-25 16:05:56.352515.json',
 'xgb_all_feat_2023-05-24 05:08:29.718082.json']

In [45]:
cat_pred = pd.read_csv('../Data/results/catboost_all_feat_2023-05-25 11:41:56.839795.csv',sep='\t')

In [46]:
xgb_pred = pd.read_csv('../Data/results/xgb_all_feat_2023-05-24 05:08:29.718082.csv',sep='\t')

In [58]:
out = np.abs(cat_pred[TrainSplit.IS_INSTALLED[0]]-xgb_pred[TrainSplit.IS_INSTALLED[0]])

In [61]:
len(out[out>0.2])

1114